In [3]:
## Copyright Mechanisms Underlying Behavior Lab, Singapore
## https://mechunderlyingbehavior.wordpress.com/

## curve_gen.py is part of the locomotion package comparing animal behaviours, developed
## to support the work discussed in the paper "Computational geometric tools for
## modeling inherent variability in animal behavior" by MT Stamps, S Go, and AS Mathuru.

## This python script contains methods for generating random curves in R^2. The closed
## of these curves will then be used to capture these curves in different frame rates
## and resolutions in the accompanying script "capture_and_compare.py" for the purpose of
## testing the robustness of our package. The mathematical basis for this curve generation
## described in the paper "Random space and plane curves" by Igor Rivin, which can be
## accessed here: https://arxiv.org/pdf/1607.05239.pdf. The main idea is to use the fourier
## series described in the paper to generate a random sequence of numbers in each coordinate
## of the plane. The resulting parametric curve of the plane f(theta) = [x(theta), y(theta)] in 
## R^2 is guaranteed to be smooth, random (in the sense that the probability of getting one curve is
## the same as getting any other) and closed (it loops back to itself when the theta = 2pi). This
## gives us random trajectories of animals that we can use to test the robustness of the package.
## To avoid getting closed-loop trajectories, we pick domains that are proper subsets of [0, 2pi].

import os
import math
import random
import numpy as np
import pandas as pd
#abc

PATH_TO_DATA_DIRECTORY = os.getcwd() + "/data"
try: # Safety check to ensure that folder exists, and makes it otherwise.
    os.mkdir(PATH_TO_DATA_DIRECTORY)
except FileExistsError:
    pass

#static variables for curve generation
NUM_CURVES = 50
NUM_TERMS = 50
POWER_OF_TERMS = 2.1
SIZE = 100
#IT'S NOT CLEAR WHAT NUM_TERMS, POWER_OF_TERMS AND SIZE ARE (MAYBE WE COULD REFER TO THE FORMULA HERE?), A SHORT EXPLANATION WOULD BE HELPFUL

#prescribed number of digits for file-naming
ZFILL_LEN = int(np.ceil(np.log10(NUM_CURVES)))


########################################################################
#### Utility Functions ####
########################################################################


def coeffGen(k, p):
    """ Generates the sequence of coefficients a_k/b_k for k = [0, no_of_terms] 
        for the fourier series in each variable. This is done by sampling from a 
        Gaussian distribution with mean 0 and a decaying standard deviation.

        :Parameters:
            k : int. The number of terms we want to sum up in the fourier series
            p : float. The power of denominator when sampled from the Gaussian distribution

        :Returns:
            list of floats. The sequence of coefficients for the sine/cosine terms
            in the fourier series for each coordinate variable.
    """
    
    #initialise return list
    coeffSeq = []

    #append a random sample from the Gaussian distribution with mean 0 and decaying standard deviation k times
    for i in range(k):
        #the power of the denominator in the is offset by 1 since range(k) is 0-indexed
        coeffSeq.append(np.random.normal(0, 1.0 / np.power(i + 1, p)))

    return coeffSeq

#LOOKS LIKE THE WAY k IS BEING USED HERE IS NOT CONSISTENT WITH WHAT k REFERS TO IN THE PAPER, I THINK IN GENERAL IT WOULD BE HELPFUL TO USE THE SAME NAMING CONVENTION AS IN THE PAPER SO THAT THE READER CAN UNDERSTAND HOW THESE FUNCTIONS REPRESENT RESULTS FROM THE PAPER
#RETURNS NOT VERY CLEAR, IS IT SUPPOSED TO BE THE SEQUENCE [a_0, a_1, ..., a_k]?
#WHERE DO WE GET P FROM?
#"a_k/b_k for k = [0, no_of_terms]", BUT k, AS DEFINED IN PARAMETERS, REFERS TO THE NUMBER OF TERMS RATHER THAN A COUNTER, NO? ALSO TO BE PAINFULLY PRECISE k IS NOT EQUAL TO AN INTERVAL BUT RATHER BELONGS TO AN INTERVAL [0, no_of_terms], NO?


########################################################################
#### Curve Generating ####
########################################################################


def genCurve(k, p, size, file_no):
    """ Writes the generated coefficients and other relevant data for the curve into a 
        csv file, whose filename corresponds to the file number provided.

        :Parameters:
            k : int. The number of terms we want to sum up in the fourier series
            p : float. The power of denominator when sampled from the Gaussian distribution
            size : int. The size of the boundary box that the resulting generated curve will be contained in
            file_no: int. Index used for file naming

        :Returns:
            dataframe with columns [a_k, b_k, c_k, d_k, extras]
            The first 4 columns will have k rows - these are the the coefficients used to generate parametric plane curve. 
            The extras column will have 6 rows corresponding to theta, size, x_min, x_max, y_min and y_max.
            This is extra data that will help with curve capturing in capture_and_compare.py.

            This dataframe will be written into the file coefficients_(file_number).csv in the curve_data folder.
    """

    #generate coefficients for the fourier series. a_k and b_k are used for the x coordinate,
    #whereas c_k and d_k are used for the y-coordinate. The dimension of each sequence is 1 x K.
    a_k = coeffGen(k, p)
    b_k = coeffGen(k, p)
    c_k = coeffGen(k, p)
    d_k = coeffGen(k, p)

    #generate a random value of theta that is less than 2pi in radians
    theta = random.random()**0.5 * (2*math.pi)
    #create bounding box that is a reasonably-sized proper subset of the given size
    x_min = size / 2.0 * random.random()**2
    x_max = size - size / 2.0 * random.random()**2
    y_min = size / 2.0 * random.random()**2
    y_max = size - size / 2.0 * random.random()**2

    #transform the coefficient and extras data into a dataframe
    data = np.transpose(np.array((a_k, b_k, c_k, d_k)))
    df1 = pd.DataFrame(data, columns = ['a_k', 'b_k', 'c_k', 'd_k'])
    extras = np.transpose(np.array([theta, size, x_min, x_max, y_min, y_max]))
    df2 = pd.DataFrame(extras, columns = ['extras'])
    df = pd.concat([df1,df2], axis = 1)

    #write the dataframe to a csv file in the curve_data directory
    dirPath = PATH_TO_DATA_DIRECTORY + "/curve_data"
    try: #safety check to ensure that folder exists, and makes it otherwise.
        os.mkdir(dirPath)
    except FileExistsError:
        pass
    curve_file_csv = dirPath + "/coefficients_{}.csv".format(str(file_no).zfill(ZFILL_LEN))
    df.to_csv(curve_file_csv, index = False)
    return df

In [4]:
def genNCurves(n, k, p):
    """ Write multiple curves, with their corresponding coefficients and extras, into multiple csv files.

        :Parameters:
            n : int. The number of curves we want to generate.
            k : int. The maximum number of terms in the trig series for any of the curves generated.
            p : float. The power of denominator when sampled from the Gaussian distribution.

        :Returns:
            None.
            Writes the files coefficients_00.csv, ..., coefficients_(n-1).csv into the data/curve_data folder.
    """

    for i in range(n):
        #uniformly sample k from a range of 2 to 100 to get a bigger variety of curves
        k = random.randrange(2,100)
        #use grid size 100x100
        genCurve(k, p, 100, i)

#generate N curves using the static variables defined above
genNCurves(NUM_CURVES, NUM_TERMS, POWER_OF_TERMS)


In [5]:
## This python script contains methods capturing and comparing randomly generated curves
## from curve_gen.py. The mathematical basis for this curve generation is described in
## the paper "Random space and plane curves" by Igor Rivin, which can be accessed here:
## https://arxiv.org/pdf/1607.05239.pdf. This script first extracts the coefficients from
## the files generated by curve_gen.py. It then generates the plane curves with coordinates
## [x(theta), y(theta)] given each step theta, and captures it based on the given framerates
## and pixel density. After capturing these test files, we run robustness tests using the
## locomotion package.


import os
import re
import sys
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

In [7]:
import locomotion

In [8]:
PATH_TO_DATA_DIRECTORY = os.getcwd() + "/data"
try: # Safety check to ensure that data folder exists, and makes it otherwise.
    os.mkdir(PATH_TO_DATA_DIRECTORY)
except FileExistsError:
    pass

PATH_TO_RES_DIRECTORY = os.getcwd() + "/results"
try: # Safety check to ensure that results folder exists, and makes it otherwise.
    os.mkdir(PATH_TO_RES_DIRECTORY)
except FileExistsError:
    pass

PATH_TO_FIG_DIRECTORY = os.getcwd() + "/figures"
try: # Safety check to ensure that figures folder exists, and makes it otherwise.
    os.mkdir(PATH_TO_FIG_DIRECTORY)
except FileExistsError:
    pass


#static variables used for robustness testing
NUM_CURVES = 50 # This must match the number of curves in the data/curve_data directory.
ZFILL_LEN = int(np.ceil(np.log10(NUM_CURVES)))
NUM_SAMPLES = 50 # Number of samples being tested
SAMP_FILL = int(np.ceil(np.log10(NUM_SAMPLES)))
DEFAULT_START = 0 # Start Time in Minutes
DEFAULT_STOP = 1 # Stop Time in Minutes

In [9]:
########################################################################
#### Functions for getting curve data  ####
########################################################################

def genTrigFun(a_k, b_k):
    """ Generates the Fourier series function f(theta) = sum_0^k (a_k * cos(k * theta) + b_k * sin(k * theta))
        based on the given coefficient sequences.

        :Parameters:
            a_k/b_k : float list. Coefficient sequences of length k.

        :Returns:
            function f(theta), which is defined by f(theta) = sum_0^k (a_k * cos(k * theta) + b_k * sin(k * theta)).
    """

    #define the function we want to return
    def fun_theta (theta):
        #initialise the sum of all k terms
        cum_sum = 0

        #add each term of the function to the sum iteratively
        for i in range(len(a_k)):
            cum_sum += a_k[i] * np.sin(i * theta) + b_k[i] * np.cos(i * theta)
        return cum_sum

    return (fun_theta)

In [13]:
def changePixDensity(num, density):
    """ Converts a coordinate given in mm (num) to the corresponding coordinate in pixels, given the pixel density.

        :Parameters:
            num : float. Coordinate value, in mm.
            density: float. Pixel Density of given file, in px/mm.

        :Returns:
            float. Corresponding coordinate value, in px.
    """
    #initialise return value
    result = 0

    #if our values aren't edge cases, return the corresponding value discretised to the given pixel density
    if not (num == 0 or density == 0 or math.isnan(num) or math.isnan(density)):
        result = math.floor(num * density)

    return result


def genVariables(low, high, n):
    """ Uniformly samples n values from given interval.

        :Parameters:
            low : float. Lower bound of interval.
            high : float. Upper bound of interval.
            n : int. Number of samples.

        :Returns:
            list of n floats. The uniformly sampled values within the interval.
    """
    return list(np.random.uniform(low, high, n))

In [14]:
########################################################################
#### Capturing curves from given frame rate/resolution  ####
########################################################################


def cameraFunc(coeff_path, time_start, time_stop, frame_rate, density, plot=False):
    """ Given a path to coefficients for the plane curve and other necessary information,
        generate curve functions and capture curves as though captured by a camera, in pixels and frames.

        :Parameters:
            coeff_path : string. Path to coefficients_xx.csv for the curve we want to capture.
            time_start, time_stop : floats. Beginning and end times to generate time step increments, in seconds.
            frame_rate : int. Number of frames per second. Used to generate the time step increments, in frames/s.
            density : float. Pixel density. Used to convert from mm to pixels.

        :Returns:
            tuple of dataframes (coordinates, summaryStats). 
            dataframe with columns [X, Y].
    """
    #NOT CLEAR WHETHER 1, 2, OR 3 DATAFRAMES ARE TO BE GENERATED (IT'S COORDINATES THAT HAS X AND Y COLUMNS RIGHT?)
    
    #read in data from the corresponding coefficients csv file
    data = pd.read_csv(coeff_path)

    #get curve number from the path
    curve_no = re.search('coefficients_(\d+)\.csv', coeff_path).group(1)

    #coefficients - each are sequences of length k
    #a_k, b_k are used for x(theta) and c_k, d_k are used for y(theta)
    a_k = data['a_k'].values
    b_k = data['b_k'].values
    c_k = data['c_k'].values
    d_k = data['d_k'].values

    #extras
    #maximum theta value for input into the trig function
    max_theta = data['extras'][0]
    #size is the full dimensions of the camera, whereas x/y min/max/diff are the corresponding
    #dimensions for the bounding box that will contain the curve
    
    #MAYBE ADD THAT THESE ARE THE DIMENSIONS BEFORE THE CONVERSION TO PIXELS?
    
    size = data['extras'][1]
    x_min = data['extras'][2]
    x_max = data['extras'][3]
    x_diff = x_max - x_min
    y_min = data['extras'][4]
    y_max = data['extras'][5]
    y_diff = y_max - y_min

    #calculate total frames
    total_frames = (time_stop - time_start) * frame_rate
    #partition the interval [0, max_theta] into total_frame steps
    thetas = max_theta * np.arange(0, 1, 1/total_frames)
    #MAYBE CLARIFY THAT THIS IS DONE IN ORDER TO OBTAIN A SEQUENCE OF THETAS (AN INTERVAL WITH DISCRETE POINTS) CORRESPONDING TO CAMERA FRAMES SO THAT FOR EACH THETA X AND Y COORDINATES CAN BE CALCULATED?

    #generate trig functions for x, y coordinates. Each function's domain is theta in [0, 2pi]
    x_fun = genTrigFun(a_k, b_k)
    y_fun = genTrigFun(c_k, d_k)

    #get minimum and maximum x, y coordinates of the graph 
    #ALSO, FIRSTLY: GET ALL X AND Y COORDINTES CAPTURED BY CAMERA (DEPENDING ON THE NUMBER OF FRAMES) YET BEFORE CONVERSION TO PIXELS
    x_og = x_fun(thetas)
    y_og = y_fun(thetas)
    lower_xlim = min(x_og)
    upper_xlim = max(x_og)
    lower_ylim = min(y_og)
    upper_ylim = max(y_og)

    #transform the coordinates to fit into the generated bounding box
    #MAYBE ADD THAT IT'S JUST TRANSLATION AND STRETCHING SO THE SHAPE SHOULDN'T BE DISTORTED?
    x_enlarged = []
    y_enlarged = []
    for i in range(0, total_frames):
        x_enlarged.append((x_diff / (upper_xlim - lower_xlim)) * (x_og[i] - lower_xlim) + x_min)
        y_enlarged.append((y_diff / (upper_ylim - lower_ylim)) * (y_og[i] - lower_ylim) + y_min)

    #transform coordinates from mm to px
    x = []
    y = []
    for i in range(0, total_frames):
        x.append(changePixDensity(x_enlarged[i], density))
        y.append(changePixDensity(y_enlarged[i], density))

    if plot:
        # Plots the X, Y coordinates and coefficients of each graph and saves it into the figures folder
        #MAYBE MENTION THAT THE PLOT X AND Y COORDINATES ARE NOW IN PIXELS?
        newsize = changePixDensity(size, density)
        plt.subplots_adjust(left = None, bottom = None, right = None, top = None, wspace = 0.5, hspace = 0.5)
        plt.subplot(121)
        plt.plot(x, y)
        plt.title("Coordinate plot for Curve No. " + curve_no)
        plt.axis([0, newsize, 0, newsize])
        plt.subplot(122)
        kSeq = np.arange(0, len(a_k), 1)
        plt.plot(kSeq, a_k)
        plt.plot(kSeq, b_k)
        plt.plot(kSeq, c_k)
        plt.plot(kSeq, d_k)
        plt.title("Coefficients of Curve No. " + curve_no)
        plt.axis([0, 20, -1, 1])
        plt.savefig(PATH_TO_FIG_DIRECTORY + "/plot_" + curve_no)
        plt.clf()

    # Transform data into dataframe
    data = np.transpose(np.array((x, y)))
    coordinates = pd.DataFrame(data, columns = ['X', 'Y'])
    summaryStats = coordinates.describe()
    return coordinates, summaryStats

In [15]:
def captureOneCurve(dat_path, curve_str, test_str, coeff_path,
                    frame_rate, density, control = "False", plot = False):
    """ Given a path to curve data, capture the corresponding curve using cameraFunc,
        and outputs the data to data_path. Then, produce the json that captures the necessary information.

        :Parameters:
            dat_path : str. Absolute file output path.
            curve_str : str. ID of the curve.
            test_str : str. ID of the test.
            coeff_path : str. Absolute file path of the respective curve coefficients.
            frame_rate : int. Framerate of file, in frames/second.
            density : int. Pixel density of file, in pixels/mm.
            control : str. Indicating whether the current test is the control.
                           Valid options = "True", "False". Default = "False".
            plot : bool. If True, will save a plot of the curve. Default = False.
        :Returns:
            jsonItem : dict. Json format, as needed in animal.py.
    """
    #MAYBE ALSO MENTION THAT THE DATAFRAME GENERATED BY CAMERAFUNC IS NOW BEING CONVERTED TO CSV AND SAVED IN THE OUTPUT DIRECTORY?

    # Generate Capture Data
    df, _ = cameraFunc(coeff_path, DEFAULT_START * 60, DEFAULT_STOP * 60, frame_rate, density, plot)
    # Save Capture Data to CSV
    df.to_csv(dat_path)
    jsonItem = {
        "name": "CRV_{}_TEST_{}".format(curve_str, test_str),
        "data_file_location": dat_path,
        "animal_attributes":
            {
                "species": "Magic Scoliosis Fish",
                "exp_type": "MCS",
                "ID": curve_str,
                "control_group": control
            },
            "capture_attributes":
            {
                "dim_x": 100,
                "dim_y": 100,
                "pixels_per_mm": density,
                "frames_per_sec": frame_rate,
                "start_time": DEFAULT_START,
                "end_time": DEFAULT_STOP,
                "baseline_start_time": DEFAULT_START,
                "baseline_end_time": DEFAULT_STOP
            }
    }
    return jsonItem

#MAYBE SPECIFY WHERE WE GOT X AND Y DIM FROM, THEY ARE THE ORIGINAL DIMENSIONS OF EACH CURVE (IN MM) BEFORE CONVERSION?

def captureAllCurves(test_key):
    """ Given a key in the testData dictionary (defined below), it iterates through the curve coefficients
        and captures the curves based on the different variables stored in testData.

        :Parameters:
            test_key : str. Must coincide with a key in testData.

        :Returns:
            None.
            Writes the file Results_variables.json into the results directory.
    """
    # Check / Create directory
    #WHERE IS PATH_TO_RES_DIRECTORY TAKEN FROM? FIRST FEW LINES OF THE SCRIPT
    resultPath = PATH_TO_RES_DIRECTORY + "/" + test_key
    try:
        os.mkdir(resultPath)
    except FileExistsError:
        pass
    for curve_no in range(NUM_CURVES):
        curve_str = str(curve_no).zfill(ZFILL_LEN)
        jsonItems = []
        coeff_path = PATH_TO_DATA_DIRECTORY + "/curve_data/coefficients_{}.csv".format(curve_str)
        # Capture Control + Key Check
        try:
            control_fr, control_dens = testData[test_key]["control"]
        except KeyError:
            raise Exception("test_key not in testData")
        control_dat = resultPath + "/CRV_{}_TEST_CTRL.dat".format(curve_str)
        control_json = captureOneCurve(control_dat, curve_str, "CTRL", coeff_path, control_fr, control_dens, "True")
        jsonItems.append(control_json)
        # Capture test curves
        i = 0
        for fr in testData[test_key]["framerates"]:
            for dens in testData[test_key]["densities"]:
                test_str = str(i).zfill(SAMP_FILL)
                dat_path = resultPath + "/CRV_{}_TEST_{}.dat".format(curve_str, test_str)
                jsonItem = captureOneCurve(dat_path, curve_str, test_str, coeff_path, fr, dens)
                jsonItems.append(jsonItem)
                i += 1
        outfilename = resultPath + "/CRV_{}.json".format(curve_str)
        jsonstr = json.dumps(jsonItems, indent = 4)
        with open(outfilename, "w") as outfile:
            outfile.write(jsonstr)
        print("Wrote the information into %s" % outfilename)
    # Save Frame Rate data and Density data
    with open(resultPath + "/Results_variables.json", "w") as outfile:
        varJson = json.dumps(testData[test_key])
        outfile.write(varJson)
       
    #I THINK THIS FUNCTION NEEDS MORE DOCUMENTATION, IT'S NOT VERY CLEAR WHAT TEST_DATA AND TEST_KEY ARE AND WHAT FILES ARE WRITTEN, ALSO IT WOULD BE HELPFUL TO ELABORATE ON WHAT THE FOR-LOOP WITHIN A FOR-LOOP ABOVE IS DOING
    



In [16]:
def runRobustnessTest(test_key, variables, norm_mode, start_min, end_min):
    """ Calculates the BDD of a test curve to the control curve of the test.
        Saves the results into a csv file in the result directory.

        :Parameters:
            test_key: str. Must coincide with a key in testData,
            variables: list of str. List of variables to use for generating the BDD.
            norm_mode: str. Normalization mode. Options defined in animal.py.
            start_min: float. Starting time in minutes.
            end_min: float. Ending time in minutes.

        :Returns:
            None.
            Writes the file Results_BDD.csv to the result directory.
    """
    #MAYBE CLARIFY WHAT THE CSV FILE SHOULD LOOK LIKE?
    
    NUM_TESTS = len(testData[test_key]["framerates"]) * len(testData[test_key]["densities"])
    results = np.zeros([NUM_CURVES, NUM_TESTS])
    for curve_no in range(NUM_CURVES):
        curve_str = str(curve_no).zfill(ZFILL_LEN)
        json_path = PATH_TO_RES_DIRECTORY + "/{}/CRV_{}.json".format(test_key, curve_str)
        # Load all animals
        animals = locomotion.getAnimalObjs(json_path)
        for a in animals:
            locomotion.trajectory.getCurveData(a)
        # Run BDD against control animal (index 0)
        control = animals[0]
        for a_no, a in enumerate(animals[1:]):
            bdd = locomotion.trajectory.computeOneBDD(a, control, variables,
                                                      start_min, end_min,
                                                      start_min, end_min,
                                                      norm_mode)
            results[curve_no][a_no] = bdd
    output = PATH_TO_RES_DIRECTORY + "/{}/Results_BDD.csv".format(test_key)
    pd.DataFrame(results).to_csv(output, index = False)

In [17]:
testData = {
    "FR_test_lower" : {
        "framerates" : list(range(6,24)),
        "densities" : [2],
        "control" : (24, 2)
    },
    "FR_test_higher" : {
        "framerates" : list(range(24,120,2)),
        "densities" : [2],
        "control" : (24, 2)
    },
    "density_test_lower" : {
        "framerates" : [24],
        "densities" : genVariables(0.5, 2, NUM_SAMPLES),
        "control" : (24, 2)
    },
    "density_test_higher" : {
        "framerates" : [24],
        "densities" : genVariables(2, 8, NUM_SAMPLES),
        "control" : (24, 2)
    }
}

# Adjust these variables to the specific test you want to run.
test_name = "FR_test_higher"
test_variables = ['Velocity', 'Curvature']
test_norm_mode = 'spec'

captureAllCurves(test_name) # Uncomment to recapture curves
runRobustnessTest(test_name, test_variables, test_norm_mode, DEFAULT_START, DEFAULT_STOP)

Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_00.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_01.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_02.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_03.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_04.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_05.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_higher/CRV_06.json
Wrote 

LOG: Extracting coordinates for Animal CRV_00_TEST_34...
LOG: Extracting coordinates for Animal CRV_00_TEST_35...
LOG: Extracting coordinates for Animal CRV_00_TEST_36...
LOG: Extracting coordinates for Animal CRV_00_TEST_37...
LOG: Extracting coordinates for Animal CRV_00_TEST_38...
LOG: Extracting coordinates for Animal CRV_00_TEST_39...
LOG: Extracting coordinates for Animal CRV_00_TEST_40...
LOG: Extracting coordinates for Animal CRV_00_TEST_41...
LOG: Extracting coordinates for Animal CRV_00_TEST_42...
LOG: Extracting coordinates for Animal CRV_00_TEST_43...
LOG: Extracting coordinates for Animal CRV_00_TEST_44...
LOG: Extracting coordinates for Animal CRV_00_TEST_45...
LOG: Extracting coordinates for Animal CRV_00_TEST_46...
LOG: Extracting coordinates for Animal CRV_00_TEST_47...
LOG: Applying DTW to the data from files CRV_00_TEST_00 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_00 and CRV_00_TEST_CTRL: 0.00000
LOG: Applying DTW to the data from files CRV_00_TEST_01

LOG: Extracting coordinates for Animal CRV_01_TEST_23...
LOG: Extracting coordinates for Animal CRV_01_TEST_24...
LOG: Extracting coordinates for Animal CRV_01_TEST_25...
LOG: Extracting coordinates for Animal CRV_01_TEST_26...
LOG: Extracting coordinates for Animal CRV_01_TEST_27...
LOG: Extracting coordinates for Animal CRV_01_TEST_28...
LOG: Extracting coordinates for Animal CRV_01_TEST_29...
LOG: Extracting coordinates for Animal CRV_01_TEST_30...
LOG: Extracting coordinates for Animal CRV_01_TEST_31...
LOG: Extracting coordinates for Animal CRV_01_TEST_32...
LOG: Extracting coordinates for Animal CRV_01_TEST_33...
LOG: Extracting coordinates for Animal CRV_01_TEST_34...
LOG: Extracting coordinates for Animal CRV_01_TEST_35...
LOG: Extracting coordinates for Animal CRV_01_TEST_36...
LOG: Extracting coordinates for Animal CRV_01_TEST_37...
LOG: Extracting coordinates for Animal CRV_01_TEST_38...
LOG: Extracting coordinates for Animal CRV_01_TEST_39...
LOG: Extracting coordinates for

LOG: Extracting coordinates for Animal CRV_02_TEST_01...
LOG: Extracting coordinates for Animal CRV_02_TEST_02...
LOG: Extracting coordinates for Animal CRV_02_TEST_03...
LOG: Extracting coordinates for Animal CRV_02_TEST_04...
LOG: Extracting coordinates for Animal CRV_02_TEST_05...
LOG: Extracting coordinates for Animal CRV_02_TEST_06...
LOG: Extracting coordinates for Animal CRV_02_TEST_07...
LOG: Extracting coordinates for Animal CRV_02_TEST_08...
LOG: Extracting coordinates for Animal CRV_02_TEST_09...
LOG: Extracting coordinates for Animal CRV_02_TEST_10...
LOG: Extracting coordinates for Animal CRV_02_TEST_11...
LOG: Extracting coordinates for Animal CRV_02_TEST_12...
LOG: Extracting coordinates for Animal CRV_02_TEST_13...
LOG: Extracting coordinates for Animal CRV_02_TEST_14...
LOG: Extracting coordinates for Animal CRV_02_TEST_15...
LOG: Extracting coordinates for Animal CRV_02_TEST_16...
LOG: Extracting coordinates for Animal CRV_02_TEST_17...
LOG: Extracting coordinates for

LOG: distance between CRV_02_TEST_39 and CRV_02_TEST_CTRL: 0.01739
LOG: Applying DTW to the data from files CRV_02_TEST_40 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_40 and CRV_02_TEST_CTRL: 0.01728
LOG: Applying DTW to the data from files CRV_02_TEST_41 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_41 and CRV_02_TEST_CTRL: 0.01766
LOG: Applying DTW to the data from files CRV_02_TEST_42 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_42 and CRV_02_TEST_CTRL: 0.01763
LOG: Applying DTW to the data from files CRV_02_TEST_43 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_43 and CRV_02_TEST_CTRL: 0.01761
LOG: Applying DTW to the data from files CRV_02_TEST_44 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_44 and CRV_02_TEST_CTRL: 0.01772
LOG: Applying DTW to the data from files CRV_02_TEST_45 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_45 and CRV_02_TEST_CTRL: 0.01780
LOG: Applying DTW to the data from files CRV_02_TES

...

LOG: distance between CRV_46_TEST_12 and CRV_46_TEST_CTRL: 0.02796
LOG: Applying DTW to the data from files CRV_46_TEST_13 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_13 and CRV_46_TEST_CTRL: 0.03043
LOG: Applying DTW to the data from files CRV_46_TEST_14 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_14 and CRV_46_TEST_CTRL: 0.01967
LOG: Applying DTW to the data from files CRV_46_TEST_15 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_15 and CRV_46_TEST_CTRL: 0.02219
LOG: Applying DTW to the data from files CRV_46_TEST_16 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_16 and CRV_46_TEST_CTRL: 0.02145
LOG: Applying DTW to the data from files CRV_46_TEST_17 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_17 and CRV_46_TEST_CTRL: 0.02213
LOG: Applying DTW to the data from files CRV_46_TEST_18 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_18 and CRV_46_TEST_CTRL: 0.02642
LOG: Applying DTW to the data from files CRV_46_TES

LOG: distance between CRV_47_TEST_07 and CRV_47_TEST_CTRL: 0.01415
LOG: Applying DTW to the data from files CRV_47_TEST_08 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_08 and CRV_47_TEST_CTRL: 0.01468
LOG: Applying DTW to the data from files CRV_47_TEST_09 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_09 and CRV_47_TEST_CTRL: 0.01520
LOG: Applying DTW to the data from files CRV_47_TEST_10 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_10 and CRV_47_TEST_CTRL: 0.01783
LOG: Applying DTW to the data from files CRV_47_TEST_11 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_11 and CRV_47_TEST_CTRL: 0.01265
LOG: Applying DTW to the data from files CRV_47_TEST_12 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_12 and CRV_47_TEST_CTRL: 0.01216
LOG: Applying DTW to the data from files CRV_47_TEST_13 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_13 and CRV_47_TEST_CTRL: 0.01236
LOG: Applying DTW to the data from files CRV_47_TES

LOG: Extracting coordinates for Animal CRV_48_TEST_45...
LOG: Extracting coordinates for Animal CRV_48_TEST_46...
LOG: Extracting coordinates for Animal CRV_48_TEST_47...
LOG: Applying DTW to the data from files CRV_48_TEST_00 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_00 and CRV_48_TEST_CTRL: 0.00000
LOG: Applying DTW to the data from files CRV_48_TEST_01 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_01 and CRV_48_TEST_CTRL: 0.01535
LOG: Applying DTW to the data from files CRV_48_TEST_02 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_02 and CRV_48_TEST_CTRL: 0.00837
LOG: Applying DTW to the data from files CRV_48_TEST_03 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_03 and CRV_48_TEST_CTRL: 0.01164
LOG: Applying DTW to the data from files CRV_48_TEST_04 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_04 and CRV_48_TEST_CTRL: 0.00874
LOG: Applying DTW to the data from files CRV_48_TEST_05 and CRV_48_TEST_CTRL...
LOG: distance 

LOG: Extracting coordinates for Animal CRV_49_TEST_24...
LOG: Extracting coordinates for Animal CRV_49_TEST_25...
LOG: Extracting coordinates for Animal CRV_49_TEST_26...
LOG: Extracting coordinates for Animal CRV_49_TEST_27...
LOG: Extracting coordinates for Animal CRV_49_TEST_28...
LOG: Extracting coordinates for Animal CRV_49_TEST_29...
LOG: Extracting coordinates for Animal CRV_49_TEST_30...
LOG: Extracting coordinates for Animal CRV_49_TEST_31...
LOG: Extracting coordinates for Animal CRV_49_TEST_32...
LOG: Extracting coordinates for Animal CRV_49_TEST_33...
LOG: Extracting coordinates for Animal CRV_49_TEST_34...
LOG: Extracting coordinates for Animal CRV_49_TEST_35...
LOG: Extracting coordinates for Animal CRV_49_TEST_36...
LOG: Extracting coordinates for Animal CRV_49_TEST_37...
LOG: Extracting coordinates for Animal CRV_49_TEST_38...
LOG: Extracting coordinates for Animal CRV_49_TEST_39...
LOG: Extracting coordinates for Animal CRV_49_TEST_40...
LOG: Extracting coordinates for

LOG: Applying DTW to the data from files CRV_49_TEST_47 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_47 and CRV_49_TEST_CTRL: 0.01483


In [18]:
test_name = "FR_test_lower"
test_variables = ['Velocity', 'Curvature']
test_norm_mode = 'spec'

captureAllCurves(test_name) # Uncomment to recapture curves
runRobustnessTest(test_name, test_variables, test_norm_mode, DEFAULT_START, DEFAULT_STOP)

Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_00.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_01.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_02.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_03.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_04.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_05.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/FR_test_lower/CRV_06.json
Wrote the inf

LOG: distance between CRV_00_TEST_09 and CRV_00_TEST_CTRL: 0.01856
LOG: Applying DTW to the data from files CRV_00_TEST_10 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_10 and CRV_00_TEST_CTRL: 0.01672
LOG: Applying DTW to the data from files CRV_00_TEST_11 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_11 and CRV_00_TEST_CTRL: 0.02072
LOG: Applying DTW to the data from files CRV_00_TEST_12 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_12 and CRV_00_TEST_CTRL: 0.01608
LOG: Applying DTW to the data from files CRV_00_TEST_13 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_13 and CRV_00_TEST_CTRL: 0.01350
LOG: Applying DTW to the data from files CRV_00_TEST_14 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_14 and CRV_00_TEST_CTRL: 0.01720
LOG: Applying DTW to the data from files CRV_00_TEST_15 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_15 and CRV_00_TEST_CTRL: 0.01387
LOG: Applying DTW to the data from files CRV_00_TES

LOG: distance between CRV_03_TEST_06 and CRV_03_TEST_CTRL: 0.01754
LOG: Applying DTW to the data from files CRV_03_TEST_07 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_07 and CRV_03_TEST_CTRL: 0.02129
LOG: Applying DTW to the data from files CRV_03_TEST_08 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_08 and CRV_03_TEST_CTRL: 0.01533
LOG: Applying DTW to the data from files CRV_03_TEST_09 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_09 and CRV_03_TEST_CTRL: 0.01375
LOG: Applying DTW to the data from files CRV_03_TEST_10 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_10 and CRV_03_TEST_CTRL: 0.01628
LOG: Applying DTW to the data from files CRV_03_TEST_11 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_11 and CRV_03_TEST_CTRL: 0.01206
LOG: Applying DTW to the data from files CRV_03_TEST_12 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_12 and CRV_03_TEST_CTRL: 0.01483
LOG: Applying DTW to the data from files CRV_03_TES

LOG: distance between CRV_06_TEST_06 and CRV_06_TEST_CTRL: 0.01352
LOG: Applying DTW to the data from files CRV_06_TEST_07 and CRV_06_TEST_CTRL...
LOG: distance between CRV_06_TEST_07 and CRV_06_TEST_CTRL: 0.01384
LOG: Applying DTW to the data from files CRV_06_TEST_08 and CRV_06_TEST_CTRL...
LOG: distance between CRV_06_TEST_08 and CRV_06_TEST_CTRL: 0.02201
LOG: Applying DTW to the data from files CRV_06_TEST_09 and CRV_06_TEST_CTRL...
LOG: distance between CRV_06_TEST_09 and CRV_06_TEST_CTRL: 0.01502
LOG: Applying DTW to the data from files CRV_06_TEST_10 and CRV_06_TEST_CTRL...
LOG: distance between CRV_06_TEST_10 and CRV_06_TEST_CTRL: 0.01629
LOG: Applying DTW to the data from files CRV_06_TEST_11 and CRV_06_TEST_CTRL...
LOG: distance between CRV_06_TEST_11 and CRV_06_TEST_CTRL: 0.01509
LOG: Applying DTW to the data from files CRV_06_TEST_12 and CRV_06_TEST_CTRL...
LOG: distance between CRV_06_TEST_12 and CRV_06_TEST_CTRL: 0.01207
LOG: Applying DTW to the data from files CRV_06_TES

LOG: Applying DTW to the data from files CRV_09_TEST_00 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_00 and CRV_09_TEST_CTRL: 0.01485
LOG: Applying DTW to the data from files CRV_09_TEST_01 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_01 and CRV_09_TEST_CTRL: 0.03426
LOG: Applying DTW to the data from files CRV_09_TEST_02 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_02 and CRV_09_TEST_CTRL: 0.03963
LOG: Applying DTW to the data from files CRV_09_TEST_03 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_03 and CRV_09_TEST_CTRL: 0.00991
LOG: Applying DTW to the data from files CRV_09_TEST_04 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_04 and CRV_09_TEST_CTRL: 0.01759
LOG: Applying DTW to the data from files CRV_09_TEST_05 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_05 and CRV_09_TEST_CTRL: 0.01534
LOG: Applying DTW to the data from files CRV_09_TEST_06 and CRV_09_TEST_CTRL...
LOG: distance between CRV_09_TEST_06 a

...

LOG: distance between CRV_39_TEST_09 and CRV_39_TEST_CTRL: 0.01169
LOG: Applying DTW to the data from files CRV_39_TEST_10 and CRV_39_TEST_CTRL...
LOG: distance between CRV_39_TEST_10 and CRV_39_TEST_CTRL: 0.00778
LOG: Applying DTW to the data from files CRV_39_TEST_11 and CRV_39_TEST_CTRL...
LOG: distance between CRV_39_TEST_11 and CRV_39_TEST_CTRL: 0.03272
LOG: Applying DTW to the data from files CRV_39_TEST_12 and CRV_39_TEST_CTRL...
LOG: distance between CRV_39_TEST_12 and CRV_39_TEST_CTRL: 0.00851
LOG: Applying DTW to the data from files CRV_39_TEST_13 and CRV_39_TEST_CTRL...
LOG: distance between CRV_39_TEST_13 and CRV_39_TEST_CTRL: 0.01462
LOG: Applying DTW to the data from files CRV_39_TEST_14 and CRV_39_TEST_CTRL...
LOG: distance between CRV_39_TEST_14 and CRV_39_TEST_CTRL: 0.00667
LOG: Applying DTW to the data from files CRV_39_TEST_15 and CRV_39_TEST_CTRL...
LOG: distance between CRV_39_TEST_15 and CRV_39_TEST_CTRL: 0.02132
LOG: Applying DTW to the data from files CRV_39_TES

LOG: Applying DTW to the data from files CRV_42_TEST_00 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_00 and CRV_42_TEST_CTRL: 0.03769
LOG: Applying DTW to the data from files CRV_42_TEST_01 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_01 and CRV_42_TEST_CTRL: 0.03269
LOG: Applying DTW to the data from files CRV_42_TEST_02 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_02 and CRV_42_TEST_CTRL: 0.02131
LOG: Applying DTW to the data from files CRV_42_TEST_03 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_03 and CRV_42_TEST_CTRL: 0.02181
LOG: Applying DTW to the data from files CRV_42_TEST_04 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_04 and CRV_42_TEST_CTRL: 0.01698
LOG: Applying DTW to the data from files CRV_42_TEST_05 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_05 and CRV_42_TEST_CTRL: 0.03176
LOG: Applying DTW to the data from files CRV_42_TEST_06 and CRV_42_TEST_CTRL...
LOG: distance between CRV_42_TEST_06 a

LOG: distance between CRV_44_TEST_05 and CRV_44_TEST_CTRL: 0.08184
LOG: Applying DTW to the data from files CRV_44_TEST_06 and CRV_44_TEST_CTRL...
LOG: distance between CRV_44_TEST_06 and CRV_44_TEST_CTRL: 0.07983
LOG: Applying DTW to the data from files CRV_44_TEST_07 and CRV_44_TEST_CTRL...
LOG: distance between CRV_44_TEST_07 and CRV_44_TEST_CTRL: 0.07786
LOG: Applying DTW to the data from files CRV_44_TEST_08 and CRV_44_TEST_CTRL...
LOG: distance between CRV_44_TEST_08 and CRV_44_TEST_CTRL: 0.07632
LOG: Applying DTW to the data from files CRV_44_TEST_09 and CRV_44_TEST_CTRL...
LOG: distance between CRV_44_TEST_09 and CRV_44_TEST_CTRL: 0.07253
LOG: Applying DTW to the data from files CRV_44_TEST_10 and CRV_44_TEST_CTRL...
LOG: distance between CRV_44_TEST_10 and CRV_44_TEST_CTRL: 0.06622
LOG: Applying DTW to the data from files CRV_44_TEST_11 and CRV_44_TEST_CTRL...
LOG: distance between CRV_44_TEST_11 and CRV_44_TEST_CTRL: 0.06908
LOG: Applying DTW to the data from files CRV_44_TES

LOG: distance between CRV_46_TEST_16 and CRV_46_TEST_CTRL: 0.02073
LOG: Applying DTW to the data from files CRV_46_TEST_17 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_17 and CRV_46_TEST_CTRL: 0.02017
LOG: Extracting coordinates for Animal CRV_47_TEST_CTRL...
LOG: Extracting coordinates for Animal CRV_47_TEST_00...
LOG: Extracting coordinates for Animal CRV_47_TEST_01...
LOG: Extracting coordinates for Animal CRV_47_TEST_02...
LOG: Extracting coordinates for Animal CRV_47_TEST_03...
LOG: Extracting coordinates for Animal CRV_47_TEST_04...
LOG: Extracting coordinates for Animal CRV_47_TEST_05...
LOG: Extracting coordinates for Animal CRV_47_TEST_06...
LOG: Extracting coordinates for Animal CRV_47_TEST_07...
LOG: Extracting coordinates for Animal CRV_47_TEST_08...
LOG: Extracting coordinates for Animal CRV_47_TEST_09...
LOG: Extracting coordinates for Animal CRV_47_TEST_10...
LOG: Extracting coordinates for Animal CRV_47_TEST_11...
LOG: Extracting coordinates for Animal CRV_

LOG: distance between CRV_49_TEST_08 and CRV_49_TEST_CTRL: 0.01585
LOG: Applying DTW to the data from files CRV_49_TEST_09 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_09 and CRV_49_TEST_CTRL: 0.01992
LOG: Applying DTW to the data from files CRV_49_TEST_10 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_10 and CRV_49_TEST_CTRL: 0.01369
LOG: Applying DTW to the data from files CRV_49_TEST_11 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_11 and CRV_49_TEST_CTRL: 0.02107
LOG: Applying DTW to the data from files CRV_49_TEST_12 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_12 and CRV_49_TEST_CTRL: 0.01611
LOG: Applying DTW to the data from files CRV_49_TEST_13 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_13 and CRV_49_TEST_CTRL: 0.01213
LOG: Applying DTW to the data from files CRV_49_TEST_14 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_14 and CRV_49_TEST_CTRL: 0.00853
LOG: Applying DTW to the data from files CRV_49_TES

In [19]:
test_name = "density_test_higher"
test_variables = ['Velocity', 'Curvature']
test_norm_mode = 'spec'

captureAllCurves(test_name) # Uncomment to recapture curves
runRobustnessTest(test_name, test_variables, test_norm_mode, DEFAULT_START, DEFAULT_STOP)

Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_higher/CRV_00.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_higher/CRV_01.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_higher/CRV_02.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_higher/CRV_03.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_higher/CRV_04.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_higher/CRV_05.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/den

LOG: Applying DTW to the data from files CRV_00_TEST_00 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_00 and CRV_00_TEST_CTRL: 0.01553
LOG: Applying DTW to the data from files CRV_00_TEST_01 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_01 and CRV_00_TEST_CTRL: 0.01725
LOG: Applying DTW to the data from files CRV_00_TEST_02 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_02 and CRV_00_TEST_CTRL: 0.02339
LOG: Applying DTW to the data from files CRV_00_TEST_03 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_03 and CRV_00_TEST_CTRL: 0.02244
LOG: Applying DTW to the data from files CRV_00_TEST_04 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_04 and CRV_00_TEST_CTRL: 0.01953
LOG: Applying DTW to the data from files CRV_00_TEST_05 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_05 and CRV_00_TEST_CTRL: 0.02461
LOG: Applying DTW to the data from files CRV_00_TEST_06 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_06 a

LOG: Applying DTW to the data from files CRV_01_TEST_00 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_00 and CRV_01_TEST_CTRL: 0.03921
LOG: Applying DTW to the data from files CRV_01_TEST_01 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_01 and CRV_01_TEST_CTRL: 0.00669
LOG: Applying DTW to the data from files CRV_01_TEST_02 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_02 and CRV_01_TEST_CTRL: 0.01387
LOG: Applying DTW to the data from files CRV_01_TEST_03 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_03 and CRV_01_TEST_CTRL: 0.01592
LOG: Applying DTW to the data from files CRV_01_TEST_04 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_04 and CRV_01_TEST_CTRL: 0.02883
LOG: Applying DTW to the data from files CRV_01_TEST_05 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_05 and CRV_01_TEST_CTRL: 0.01530
LOG: Applying DTW to the data from files CRV_01_TEST_06 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_06 a

LOG: Extracting coordinates for Animal CRV_02_TEST_35...
LOG: Extracting coordinates for Animal CRV_02_TEST_36...
LOG: Extracting coordinates for Animal CRV_02_TEST_37...
LOG: Extracting coordinates for Animal CRV_02_TEST_38...
LOG: Extracting coordinates for Animal CRV_02_TEST_39...
LOG: Extracting coordinates for Animal CRV_02_TEST_40...
LOG: Extracting coordinates for Animal CRV_02_TEST_41...
LOG: Extracting coordinates for Animal CRV_02_TEST_42...
LOG: Extracting coordinates for Animal CRV_02_TEST_43...
LOG: Extracting coordinates for Animal CRV_02_TEST_44...
LOG: Extracting coordinates for Animal CRV_02_TEST_45...
LOG: Extracting coordinates for Animal CRV_02_TEST_46...
LOG: Extracting coordinates for Animal CRV_02_TEST_47...
LOG: Extracting coordinates for Animal CRV_02_TEST_48...
LOG: Extracting coordinates for Animal CRV_02_TEST_49...
LOG: Applying DTW to the data from files CRV_02_TEST_00 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_00 and CRV_02_TEST_CTRL: 0.0202

LOG: Applying DTW to the data from files CRV_03_TEST_00 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_00 and CRV_03_TEST_CTRL: 0.01916
LOG: Applying DTW to the data from files CRV_03_TEST_01 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_01 and CRV_03_TEST_CTRL: 0.01667
LOG: Applying DTW to the data from files CRV_03_TEST_02 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_02 and CRV_03_TEST_CTRL: 0.01537
LOG: Applying DTW to the data from files CRV_03_TEST_03 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_03 and CRV_03_TEST_CTRL: 0.02146
LOG: Applying DTW to the data from files CRV_03_TEST_04 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_04 and CRV_03_TEST_CTRL: 0.01705
LOG: Applying DTW to the data from files CRV_03_TEST_05 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_05 and CRV_03_TEST_CTRL: 0.01835
LOG: Applying DTW to the data from files CRV_03_TEST_06 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_06 a

...

LOG: Applying DTW to the data from files CRV_45_TEST_00 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_00 and CRV_45_TEST_CTRL: 0.03655
LOG: Applying DTW to the data from files CRV_45_TEST_01 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_01 and CRV_45_TEST_CTRL: 0.01688
LOG: Applying DTW to the data from files CRV_45_TEST_02 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_02 and CRV_45_TEST_CTRL: 0.08298
LOG: Applying DTW to the data from files CRV_45_TEST_03 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_03 and CRV_45_TEST_CTRL: 0.08399
LOG: Applying DTW to the data from files CRV_45_TEST_04 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_04 and CRV_45_TEST_CTRL: 0.07995
LOG: Applying DTW to the data from files CRV_45_TEST_05 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_05 and CRV_45_TEST_CTRL: 0.02531
LOG: Applying DTW to the data from files CRV_45_TEST_06 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_06 a

LOG: Applying DTW to the data from files CRV_46_TEST_00 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_00 and CRV_46_TEST_CTRL: 0.02649
LOG: Applying DTW to the data from files CRV_46_TEST_01 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_01 and CRV_46_TEST_CTRL: 0.02811
LOG: Applying DTW to the data from files CRV_46_TEST_02 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_02 and CRV_46_TEST_CTRL: 0.04141
LOG: Applying DTW to the data from files CRV_46_TEST_03 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_03 and CRV_46_TEST_CTRL: 0.03094
LOG: Applying DTW to the data from files CRV_46_TEST_04 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_04 and CRV_46_TEST_CTRL: 0.02636
LOG: Applying DTW to the data from files CRV_46_TEST_05 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_05 and CRV_46_TEST_CTRL: 0.03230
LOG: Applying DTW to the data from files CRV_46_TEST_06 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_06 a

LOG: Applying DTW to the data from files CRV_47_TEST_00 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_00 and CRV_47_TEST_CTRL: 0.01904
LOG: Applying DTW to the data from files CRV_47_TEST_01 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_01 and CRV_47_TEST_CTRL: 0.01818
LOG: Applying DTW to the data from files CRV_47_TEST_02 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_02 and CRV_47_TEST_CTRL: 0.01324
LOG: Applying DTW to the data from files CRV_47_TEST_03 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_03 and CRV_47_TEST_CTRL: 0.01267
LOG: Applying DTW to the data from files CRV_47_TEST_04 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_04 and CRV_47_TEST_CTRL: 0.01372
LOG: Applying DTW to the data from files CRV_47_TEST_05 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_05 and CRV_47_TEST_CTRL: 0.01511
LOG: Applying DTW to the data from files CRV_47_TEST_06 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_06 a

LOG: Applying DTW to the data from files CRV_48_TEST_00 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_00 and CRV_48_TEST_CTRL: 0.03013
LOG: Applying DTW to the data from files CRV_48_TEST_01 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_01 and CRV_48_TEST_CTRL: 0.01898
LOG: Applying DTW to the data from files CRV_48_TEST_02 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_02 and CRV_48_TEST_CTRL: 0.02740
LOG: Applying DTW to the data from files CRV_48_TEST_03 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_03 and CRV_48_TEST_CTRL: 0.02886
LOG: Applying DTW to the data from files CRV_48_TEST_04 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_04 and CRV_48_TEST_CTRL: 0.02230
LOG: Applying DTW to the data from files CRV_48_TEST_05 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_05 and CRV_48_TEST_CTRL: 0.02600
LOG: Applying DTW to the data from files CRV_48_TEST_06 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_06 a

LOG: Applying DTW to the data from files CRV_49_TEST_00 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_00 and CRV_49_TEST_CTRL: 0.01022
LOG: Applying DTW to the data from files CRV_49_TEST_01 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_01 and CRV_49_TEST_CTRL: 0.01670
LOG: Applying DTW to the data from files CRV_49_TEST_02 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_02 and CRV_49_TEST_CTRL: 0.01008
LOG: Applying DTW to the data from files CRV_49_TEST_03 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_03 and CRV_49_TEST_CTRL: 0.01589
LOG: Applying DTW to the data from files CRV_49_TEST_04 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_04 and CRV_49_TEST_CTRL: 0.01855
LOG: Applying DTW to the data from files CRV_49_TEST_05 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_05 and CRV_49_TEST_CTRL: 0.01031
LOG: Applying DTW to the data from files CRV_49_TEST_06 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_06 a

In [20]:
test_name = "density_test_lower"
test_variables = ['Velocity', 'Curvature']
test_norm_mode = 'spec'

captureAllCurves(test_name) # Uncomment to recapture curves
runRobustnessTest(test_name, test_variables, test_norm_mode, DEFAULT_START, DEFAULT_STOP)

Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_lower/CRV_00.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_lower/CRV_01.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_lower/CRV_02.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_lower/CRV_03.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_lower/CRV_04.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_test_lower/CRV_05.json
Wrote the information into /Users/karolinagrzeszkiewicz/robustness-tests/locomotion/tests/robustness_tests/results/density_t

LOG: Applying DTW to the data from files CRV_00_TEST_00 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_00 and CRV_00_TEST_CTRL: 0.02278
LOG: Applying DTW to the data from files CRV_00_TEST_01 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_01 and CRV_00_TEST_CTRL: 0.03240
LOG: Applying DTW to the data from files CRV_00_TEST_02 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_02 and CRV_00_TEST_CTRL: 0.04620
LOG: Applying DTW to the data from files CRV_00_TEST_03 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_03 and CRV_00_TEST_CTRL: 0.03137
LOG: Applying DTW to the data from files CRV_00_TEST_04 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_04 and CRV_00_TEST_CTRL: 0.03034
LOG: Applying DTW to the data from files CRV_00_TEST_05 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_05 and CRV_00_TEST_CTRL: 0.02610
LOG: Applying DTW to the data from files CRV_00_TEST_06 and CRV_00_TEST_CTRL...
LOG: distance between CRV_00_TEST_06 a

LOG: Applying DTW to the data from files CRV_01_TEST_00 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_00 and CRV_01_TEST_CTRL: 0.01909
LOG: Applying DTW to the data from files CRV_01_TEST_01 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_01 and CRV_01_TEST_CTRL: 0.02760
LOG: Applying DTW to the data from files CRV_01_TEST_02 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_02 and CRV_01_TEST_CTRL: 0.01199
LOG: Applying DTW to the data from files CRV_01_TEST_03 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_03 and CRV_01_TEST_CTRL: 0.01406
LOG: Applying DTW to the data from files CRV_01_TEST_04 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_04 and CRV_01_TEST_CTRL: 0.01040
LOG: Applying DTW to the data from files CRV_01_TEST_05 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_05 and CRV_01_TEST_CTRL: 0.01992
LOG: Applying DTW to the data from files CRV_01_TEST_06 and CRV_01_TEST_CTRL...
LOG: distance between CRV_01_TEST_06 a

LOG: Applying DTW to the data from files CRV_02_TEST_00 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_00 and CRV_02_TEST_CTRL: 0.02967
LOG: Applying DTW to the data from files CRV_02_TEST_01 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_01 and CRV_02_TEST_CTRL: 0.02228
LOG: Applying DTW to the data from files CRV_02_TEST_02 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_02 and CRV_02_TEST_CTRL: 0.03206
LOG: Applying DTW to the data from files CRV_02_TEST_03 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_03 and CRV_02_TEST_CTRL: 0.02595
LOG: Applying DTW to the data from files CRV_02_TEST_04 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_04 and CRV_02_TEST_CTRL: 0.02155
LOG: Applying DTW to the data from files CRV_02_TEST_05 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_05 and CRV_02_TEST_CTRL: 0.02651
LOG: Applying DTW to the data from files CRV_02_TEST_06 and CRV_02_TEST_CTRL...
LOG: distance between CRV_02_TEST_06 a

LOG: Applying DTW to the data from files CRV_03_TEST_00 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_00 and CRV_03_TEST_CTRL: 0.02861
LOG: Applying DTW to the data from files CRV_03_TEST_01 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_01 and CRV_03_TEST_CTRL: 0.02318
LOG: Applying DTW to the data from files CRV_03_TEST_02 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_02 and CRV_03_TEST_CTRL: 0.02352
LOG: Applying DTW to the data from files CRV_03_TEST_03 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_03 and CRV_03_TEST_CTRL: 0.02550
LOG: Applying DTW to the data from files CRV_03_TEST_04 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_04 and CRV_03_TEST_CTRL: 0.02057
LOG: Applying DTW to the data from files CRV_03_TEST_05 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_05 and CRV_03_TEST_CTRL: 0.02558
LOG: Applying DTW to the data from files CRV_03_TEST_06 and CRV_03_TEST_CTRL...
LOG: distance between CRV_03_TEST_06 a

...

LOG: Applying DTW to the data from files CRV_45_TEST_00 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_00 and CRV_45_TEST_CTRL: 0.04581
LOG: Applying DTW to the data from files CRV_45_TEST_01 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_01 and CRV_45_TEST_CTRL: 0.03403
LOG: Applying DTW to the data from files CRV_45_TEST_02 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_02 and CRV_45_TEST_CTRL: 0.01897
LOG: Applying DTW to the data from files CRV_45_TEST_03 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_03 and CRV_45_TEST_CTRL: 0.01027
LOG: Applying DTW to the data from files CRV_45_TEST_04 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_04 and CRV_45_TEST_CTRL: 0.07336
LOG: Applying DTW to the data from files CRV_45_TEST_05 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_05 and CRV_45_TEST_CTRL: 0.01471
LOG: Applying DTW to the data from files CRV_45_TEST_06 and CRV_45_TEST_CTRL...
LOG: distance between CRV_45_TEST_06 a

LOG: Applying DTW to the data from files CRV_46_TEST_00 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_00 and CRV_46_TEST_CTRL: 0.04592
LOG: Applying DTW to the data from files CRV_46_TEST_01 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_01 and CRV_46_TEST_CTRL: 0.03741
LOG: Applying DTW to the data from files CRV_46_TEST_02 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_02 and CRV_46_TEST_CTRL: 0.03899
LOG: Applying DTW to the data from files CRV_46_TEST_03 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_03 and CRV_46_TEST_CTRL: 0.03006
LOG: Applying DTW to the data from files CRV_46_TEST_04 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_04 and CRV_46_TEST_CTRL: 0.03471
LOG: Applying DTW to the data from files CRV_46_TEST_05 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_05 and CRV_46_TEST_CTRL: 0.04277
LOG: Applying DTW to the data from files CRV_46_TEST_06 and CRV_46_TEST_CTRL...
LOG: distance between CRV_46_TEST_06 a

LOG: Applying DTW to the data from files CRV_47_TEST_00 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_00 and CRV_47_TEST_CTRL: 0.01895
LOG: Applying DTW to the data from files CRV_47_TEST_01 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_01 and CRV_47_TEST_CTRL: 0.01666
LOG: Applying DTW to the data from files CRV_47_TEST_02 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_02 and CRV_47_TEST_CTRL: 0.01792
LOG: Applying DTW to the data from files CRV_47_TEST_03 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_03 and CRV_47_TEST_CTRL: 0.01477
LOG: Applying DTW to the data from files CRV_47_TEST_04 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_04 and CRV_47_TEST_CTRL: 0.01564
LOG: Applying DTW to the data from files CRV_47_TEST_05 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_05 and CRV_47_TEST_CTRL: 0.01971
LOG: Applying DTW to the data from files CRV_47_TEST_06 and CRV_47_TEST_CTRL...
LOG: distance between CRV_47_TEST_06 a

LOG: Extracting coordinates for Animal CRV_48_TEST_42...
LOG: Extracting coordinates for Animal CRV_48_TEST_43...
LOG: Extracting coordinates for Animal CRV_48_TEST_44...
LOG: Extracting coordinates for Animal CRV_48_TEST_45...
LOG: Extracting coordinates for Animal CRV_48_TEST_46...
LOG: Extracting coordinates for Animal CRV_48_TEST_47...
LOG: Extracting coordinates for Animal CRV_48_TEST_48...
LOG: Extracting coordinates for Animal CRV_48_TEST_49...
LOG: Applying DTW to the data from files CRV_48_TEST_00 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_00 and CRV_48_TEST_CTRL: 0.04157
LOG: Applying DTW to the data from files CRV_48_TEST_01 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_01 and CRV_48_TEST_CTRL: 0.04228
LOG: Applying DTW to the data from files CRV_48_TEST_02 and CRV_48_TEST_CTRL...
LOG: distance between CRV_48_TEST_02 and CRV_48_TEST_CTRL: 0.01855
LOG: Applying DTW to the data from files CRV_48_TEST_03 and CRV_48_TEST_CTRL...
LOG: distance between C

LOG: Applying DTW to the data from files CRV_49_TEST_00 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_00 and CRV_49_TEST_CTRL: 0.01433
LOG: Applying DTW to the data from files CRV_49_TEST_01 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_01 and CRV_49_TEST_CTRL: 0.01581
LOG: Applying DTW to the data from files CRV_49_TEST_02 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_02 and CRV_49_TEST_CTRL: 0.02036
LOG: Applying DTW to the data from files CRV_49_TEST_03 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_03 and CRV_49_TEST_CTRL: 0.02442
LOG: Applying DTW to the data from files CRV_49_TEST_04 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_04 and CRV_49_TEST_CTRL: 0.01234
LOG: Applying DTW to the data from files CRV_49_TEST_05 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_05 and CRV_49_TEST_CTRL: 0.01631
LOG: Applying DTW to the data from files CRV_49_TEST_06 and CRV_49_TEST_CTRL...
LOG: distance between CRV_49_TEST_06 a